In [1]:
import torch
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('Bank_Personal_Loan_Modelling-Copy1.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [4]:
df.head()


,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [5]:
df.drop("ZIP Code",axis=1,inplace=True)
df.drop("ID",axis=1,inplace=True)

In [6]:
df['target']=df['Personal Loan']


In [7]:
df.drop('Personal Loan',axis=1,inplace=True)

In [8]:
df["Experience"] = abs(df["Experience"])

In [9]:
df

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,target
0,25,1,49,4,1.6,1,0,1,0,0,0,0
1,45,19,34,3,1.5,1,0,1,0,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,1,1.9,3,0,0,0,1,0,0
4996,30,4,15,4,0.4,1,85,0,0,1,0,0
4997,63,39,24,2,0.3,3,0,0,0,0,0,0
4998,65,40,49,3,0.5,2,0,0,0,1,0,0


In [10]:
x = df.iloc[:,:-1].values
y= df.iloc[:,-1].values

In [11]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.3,random_state =0)

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [13]:
import torch
from torch.utils.data import DataLoader, TensorDataset
BATCH_SIZE = 2
train_x = torch.from_numpy(x_train).to(torch.float32)
train_y = torch.from_numpy(y_train).to(torch.float32)


In [14]:
data = TensorDataset(train_x,train_y)
data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)

In [33]:
test_x = torch.from_numpy(x_test).to(torch.float32)
test_y = torch.from_numpy(y_test).to(torch.float32)
test = TensorDataset(test_x,test_y)
test = DataLoader(test,batch_size=20)

In [34]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model,self).__init__()
        
        self.layer1 = torch.nn.Linear(11,4)
        self.layer2 = torch.nn.Linear(4,1)
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

In [35]:
model = Model()

In [36]:
from pyswarms.single import GlobalBestPSO
torch.set_grad_enabled(False)

param = np.concatenate([i.numpy().flatten() for i in model.parameters()])
shape = [i.numpy().shape for i in model.parameters()]
size = [i[0]*i[1] if len(i) == 2 else i[0] for i in shape]
print("Dim : ", len(param))
print("Shape of different layers : ", shape)

Dim :  53
Shape of different layers :  [(4, 11), (4,), (1, 4), (1,)]


In [37]:
def objective_function(particle,shape=shape,size=size):
    acc_score = []
    output = []
    
    for par in particle:
        param=list()
        cum_sum = 0
        for i in range(len(size)):
            array = par[cum_sum : cum_sum + size[i]]
            array = array.reshape(shape[i])
            cum_sum += size[i]
            param.append(array)
        param = np.array(param, dtype="object")
        output.append(param)
    
    for i in range(len(output)):
        # Copy Weights and Biases
        model = Model()
        for idx,weights in enumerate(model.parameters()):
            weights.data = (torch.tensor(output[i][idx])).to(torch.float32)
        
        # Calculate Accuracy
        y_pred = model(train_x)
        y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
        acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
        acc_score.append(1 - acc) # Optimization function aims to reduce the cost so (1 - accuracy)
        
    return acc_score

In [30]:
grid = {'c1': 0.6, 'c2': 0.3, 'w': 0.1}
dim = len(param)
x_max = 1.0 * np.ones(dim)
x_min = -1.0 * x_max
bounds = (x_min, x_max)

pso = GlobalBestPSO(n_particles=500, dimensions=53, options=grid, bounds=bounds)

In [31]:
best_cost, best_parameters = pso.optimize(objective_function, iters=20)
print("Accuracy : ", 1 - best_cost)

2023-04-05 23:34:11,490 - pyswarms.single.global_best - INFO - Optimize for 20 iters with {'c1': 0.6, 'c2': 0.3, 'w': 0.1}
pyswarms.single.global_best: 100%|██████████|20/20, best_cost=0.068 
2023-04-05 23:34:18,459 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.06799999999999995, best pos: [-0.14849521 -0.22940588 -0.17302398  0.23184942  0.10156161 -0.22359176
 -0.37121907 -0.09033589 -0.19846131 -0.09837623  0.02568816  0.22105808
  0.14924003 -0.02020857 -0.2035208   0.21937443  0.51062772  0.47404398
 -0.11477231  0.38696465  0.27481079 -0.15535215  0.20099448 -0.43229432
  0.31637497  0.26137001  0.37128553  0.23974473 -0.19551227  0.21137198
  0.11117782 -0.19565045  0.14858566 -0.27351595 -0.04658513 -0.26655365
  0.03614116 -0.32214922 -0.33876283  0.07139017  0.05238144  0.09003727
  0.21174496  0.05799406  0.07532056  0.37460369  0.60823298 -0.10900453
 -0.05073774  0.11223733  0.15937026 -0.36966541 -0.38360656]


Accuracy :  0.932


In [39]:
result = []
for par in [best_parameters]:
    param = list()
    cum_sum = 0
    for i in range(len(size)):

        array = par[cum_sum : cum_sum + size[i]]
        array = array.reshape(shape[i])
        cum_sum += size[i]
        param.append(array)
    param = np.array(param, dtype="object")
    result.append(param)

best_model = Model()
for idx,wei in enumerate(best_model.parameters()):
    wei.data = (torch.tensor(result[0][idx])).to(torch.float32)

# Calculate Accuracy
y_pred = best_model(train_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
print("Accuracy : ", acc)

Accuracy :  0.932


In [40]:
from sklearn.metrics import classification_report
y_pred = model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
print(classification_report(y_pred,test_y))

              precision    recall  f1-score   support

           0       0.06      0.84      0.12       105
           1       0.87      0.08      0.15      1395

    accuracy                           0.13      1500
   macro avg       0.47      0.46      0.13      1500
weighted avg       0.81      0.13      0.14      1500

